In [1]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length=2048
dtype=None
load_in_4bit=True
model,tokenizer=FastLanguageModel.from_pretrained(
    model_name='unsloth/Llama-3.2-3B-Instruct',
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

# Improve attention   
-- q_proq
-- k_proj
-- v_proj
-- o_proj
# enhancing overall understanding
-- gate_proj
# improve feedforward layer
-- up_proj
--down_proj

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],  # fixed typo
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# prepare DataSet


In [48]:
import pandas as pd
import numpy as np

In [49]:
df=pd.read_csv('/content/Eminem_Lyrics.csv',    sep='\t',
    encoding='ISO-8859-1',
    on_bad_lines='skip')

In [62]:
df = df[df['Lyrics'].notna()]
df['Lyrics'] = df['Lyrics'].str.replace(r'\[.*?\]', '', regex=True)
df['Lyrics'] = df['Lyrics'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [63]:
df['Lyrics_len']=df['Lyrics'].apply(lambda x:len(x))

In [64]:
df1=df[df['Lyrics_len']>200]

In [65]:
df1['Lyrics']

,Lyrics
1,"Black magic, night walker (Yeah) She haunts me..."
2,"Before I check the mic (Check, check, one, two..."
3,"Yeah, I'm sorry (Huh?) What did you say? Oh, I..."
4,"I don't smile, I don't frown, get too up or ge..."
5,"Nah, for real, you know what I'm sayin'? Like ..."
...,...
343,I know there's something in the wake of your s...
344,"Yeah, yeah, I get it I run this rap shit, now ..."
345,"I cut back on the syllables just a little, but..."
346,C'mon! I still remember the Hip Hop Shop days ...


In [66]:
df1.drop(columns=['Album_URL','Views','Release_date','Unnamed: 6','Lyrics_len','Album_Name'],inplace=True)
data=df1.to_dict(orient='list')

<ipython-input-66-41c025d5a12e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop(columns=['Album_URL','Views','Release_date','Unnamed: 6','Lyrics_len','Album_Name'],inplace=True)


In [67]:
def formatting_prompts_func(batch):
    # batch['Lyrics'] and batch['Song_Name'] are lists
    prompts = []
    for song_name, lyrics in zip(batch['Song_Name'], batch['Lyrics']):
        prompt = f"Write a rap like Eminem titled '{song_name}':\n{lyrics}"
        prompts.append(prompt)
    return {"texts": prompts}  # ✅ Return a list with the same length as batch


In [68]:
from datasets import load_dataset,Dataset
dataset=Dataset.from_dict(data)
dataset = dataset.map(formatting_prompts_func, batched = True,)


Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [69]:
dataset[0]

{'Song_Name': 'Black Magic',
 'Lyrics': 'Black magic, night walker (Yeah) She haunts me like no other (Feel like) Nobody told me (I don\'t know) love is pain, oh (I know we just met) Black magic, dark water (But it\'s like) Surrounds me like no other (It\'s like I know you better than anyone) She\'s got my heart in chains We\'re volatile, I can\'t call it, though It\'s like too large a peg, and too small a hole (Yeah) But she cheats and I catch her like the common cold (Sneeze) Last time, I broke her collar bone, she\'s intolerabl? (Yeah) All I know is that the sex is ph?nomenal, though We\'re an unlikely pair (Yeah) Like two different Nike Airs But I\'m the same size she wears (Jordans) So I think we\'re soulmates though (Yeah) People don\'t like us together, but like we care (Not me, not me) Yeah, but God strike me dead (Yeah) She knows that I would walk over hot coals For her with both of my feet bare (Yeah) But soon as I ask to hit (Ask to hit) It\'s like she grabs a switch (Grabs 

In [71]:
from trl import SFTTrainer
from transformers import TrainingArguments,DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

In [77]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="texts",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,

        output_dir="outputs",
        report_to="none",
    ),
)

Unsloth: Tokenizing ["texts"] (num_proc=2):   0%|          | 0/345 [00:00<?, ? examples/s]

In [75]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 345 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 12,156,928/3,000,000,000 (0.41% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,3.173400
20,2.881800
30,2.808200
40,2.785400
50,2.814900
60,2.821000


# Model Inferencing

In [83]:
prompt = f"### Instruction:\nWrite a rap like Eminem titled Enemies\n\n### Response:\n"
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=300,
    do_sample=True,  # important for creativity
    top_k=50,        # control diversity
    top_p=0.95,
    temperature=1.0,
    repetition_penalty=1.2,
)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


### Instruction:
Write a rap like Eminem titled Enemies

### Response:
It's so cold in this part of town now and the snow just keeps on fallin' And I can't even afford an umbrella to protect my damn face The news has been bad lately I'm startin', talkin',"The world is on fire", what does it mean? "They don't care about us, but that's all right" You know we've had enough of being blackballed 'til your head pops That's why you won't find me at parties with these hoopers And if I go off my roof, I'll get somebody dead If everybody who'd rather have a gun than be in pain Would pull out one up their ass and pull his trigger They say some niggas don't want peace But the way they act sometimes makes the most likely man wanna seize the last word They might see the whole nation goin' apeshit One day when every single song is just gonna cut deep into ya chest shit How long have there been wars over people bein' different? We're fightin' over land that ain't even fertile I'm not racist against th

# saving the Model


In [84]:
model.save_pretrained('rap_model')
tokenizer.save_pretrained('rap_model')

('rap_model/tokenizer_config.json',
 'rap_model/special_tokens_map.json',
 'rap_model/tokenizer.json')

In [98]:
from google.colab import userdata
token=userdata.get('huggingface_token')


In [101]:
model.push_to_hub('ababhay08/em_rap_model',token=token)
tokenizer.push_to_hub('ababhay08/em_rap_model',token=token)

No files have been modified since last commit. Skipping to prevent empty commit.


Saved model to https://huggingface.co/ababhay08/em_rap_model


No files have been modified since last commit. Skipping to prevent empty commit.
